In [1]:
from cstm.database_helpers import get_db_connection, table_name
from cstm.query import *

In [2]:
database_path = r"database/database.db"

In [3]:
cur = get_db_connection(database_path).cursor()

In [59]:
temp_query = f"SELECT company, ticker, id, member_name, transaction_date, " \
             f"CASE transaction_type WHEN 'P' THEN value_lb ELSE 0 END AS purchase_lb, "\
             f"CASE transaction_type WHEN 'P' THEN value_ub ELSE 0 END AS purchase_ub, "\
             f"CASE transaction_type WHEN 'S' THEN value_lb ELSE 0 END AS sale_lb, "\
             f"CASE transaction_type WHEN 'S' THEN value_ub ELSE 0 END AS sale_ub "\
             f"FROM {table_name} "
full_query = f"WITH temp as ({temp_query}) "\
             f"SELECT "\
             f"COUNT(id) AS num_transactions, "\
             f"COUNT(DISTINCT member_name) AS num_members, "\
             f"SUM(purchase_lb) as purchase_lb, "\
             f"SUM(purchase_ub) as purchase_ub, "\
             f"SUM(sale_lb) as sale_lb, "\
             f"SUM(sale_ub) as sale_ub, "\
             f"strftime(\'%d-%m-%Y\', transaction_date) as transaction_date "\
             f"FROM temp " \
             f"WHERE transaction_date < \'2022-01-01\'" \
             f"GROUP BY transaction_date"

In [60]:
data = cur.execute(full_query)
column_names = [col[0] for col in data.description]
data = data.fetchall()

In [56]:
from datetime import datetime


def convert_date_string_lst_to_datetime(string_lst: list[str], str_format: str) -> list[datetime]:
    return [datetime.strptime(date_str, str_format) for date_str in string_lst]

In [91]:
print(column_names)

['num_transactions', 'num_members', 'purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub', 'transaction_date']


In [61]:
transaction_date = [row['transaction_date'] for row in data]
print(transaction_date)
purchase_lb = [row['purchase_lb'] for row in data]
purchase_ub = [row['purchase_ub'] for row in data]
sale_lb = [-row['sale_lb'] for row in data]
sale_ub = [-row['sale_ub'] for row in data]
transaction_date = convert_date_string_lst_to_datetime(transaction_date, '%d-%m-%Y')
print(transaction_date)

['26-12-2013', '16-01-2014', '06-02-2014', '10-03-2014', '05-05-2014', '22-05-2014', '11-09-2014', '11-12-2014', '16-12-2014', '17-12-2014', '18-12-2014', '24-12-2014', '25-02-2015', '25-03-2015', '26-05-2015', '19-06-2015', '20-08-2015', '24-08-2015', '04-12-2015', '17-12-2015', '29-12-2015', '30-12-2015', '25-01-2016', '27-01-2016', '02-02-2016', '07-09-2016', '17-03-2017', '29-08-2017', '28-11-2018']
[datetime.datetime(2013, 12, 26, 0, 0), datetime.datetime(2014, 1, 16, 0, 0), datetime.datetime(2014, 2, 6, 0, 0), datetime.datetime(2014, 3, 10, 0, 0), datetime.datetime(2014, 5, 5, 0, 0), datetime.datetime(2014, 5, 22, 0, 0), datetime.datetime(2014, 9, 11, 0, 0), datetime.datetime(2014, 12, 11, 0, 0), datetime.datetime(2014, 12, 16, 0, 0), datetime.datetime(2014, 12, 17, 0, 0), datetime.datetime(2014, 12, 18, 0, 0), datetime.datetime(2014, 12, 24, 0, 0), datetime.datetime(2015, 2, 25, 0, 0), datetime.datetime(2015, 3, 25, 0, 0), datetime.datetime(2015, 5, 26, 0, 0), datetime.datetime(

In [62]:
import plotly.express as px
import plotly.graph_objs as go
import pandas
from pandas import DataFrame

In [63]:
purchase_sales_lb_ub_df = DataFrame([transaction_date, purchase_lb, purchase_ub, sale_lb, sale_ub]).transpose()
purchase_sales_lb_ub_df.columns = ['transaction_date', 'purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub']

In [35]:
print(purchase_sales_lb_ub_df.columns)

Index(['transaction_date', 'purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub'], dtype='object')


In [64]:
purchase_lb_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                                 y=purchase_sales_lb_ub_df['purchase_lb'],
                                 name='purchase_lb',
                                 texttemplate="%{x}", textfont_size=20)
purchase_ub_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                                 y=purchase_sales_lb_ub_df['purchase_ub'],
                                 name='purchase_ub',
                                 texttemplate="%{x}", textfont_size=20)
sale_lb_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                             y=purchase_sales_lb_ub_df['sale_lb'],
                             name='sale_lb',
                             texttemplate="%{x}", textfont_size=20)
sale_ub_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                             y=purchase_sales_lb_ub_df['sale_ub'],
                             name='sale_ub',
                             texttemplate="%{x}", textfont_size=20)

In [65]:
fig = go.Figure()
fig.add_trace(purchase_ub_graph)
fig.add_trace(purchase_lb_graph)
fig.add_trace(sale_ub_graph)
fig.add_trace(sale_lb_graph)
fig.update_layout(barmode='group')
fig.update_traces(opacity=0.5)
fig.show()

In [66]:
fig_alt = px.line(purchase_sales_lb_ub_df, x = 'transaction_date', y = ['purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub'])
fig_alt.update_traces(mode='markers+lines')
fig_alt.show()


In [68]:
fig_third = px.histogram(purchase_sales_lb_ub_df, x = 'transaction_date', y=purchase_sales_lb_ub_df.columns, barmode='overlay')
fig_third.show()